<a href="https://colab.research.google.com/github/Nasreen69/NLP/blob/main/IMDB_Reviews_Text_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy 
import string
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC 
from sklearn.pipeline import Pipeline
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
!wget https://www.dropbox.com/s/dctsk9k67x2jgnb/imdb_labelled.txt

--2022-01-01 13:44:05--  https://www.dropbox.com/s/dctsk9k67x2jgnb/imdb_labelled.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dctsk9k67x2jgnb/imdb_labelled.txt [following]
--2022-01-01 13:44:05--  https://www.dropbox.com/s/raw/dctsk9k67x2jgnb/imdb_labelled.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb619e8cf347e50e3ac8d61f259.dl.dropboxusercontent.com/cd/0/inline/Bc_LVCt3ATSBI5ga2kd92f9S6zthu-Ni1IlLQ2TRRVYUEQdmzmKEbOajDaSCH2K0SE5k0jyMwcdjqbTyGq4oeBzyJrT-Fq-G9ixmiXx9Xm3MJlO4kZLhj1LdUMetmOv880YlkAlH3fAnMDQRKASBkTek/file# [following]
--2022-01-01 13:44:06--  https://ucb619e8cf347e50e3ac8d61f259.dl.dropboxusercontent.com/cd/0/inline/Bc_LVCt3ATSBI5ga2kd92f9S6zthu-Ni1IlLQ2TRRVYUEQdmzmKEbOajDaSCH2K0SE5k0jyM

In [ ]:
df = pd.read_csv("/content/imdb_labelled.txt", sep = '\t', header= None )
df.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
df.columns = ['Review', 'Sentiment']
df.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
punct = string.punctuation

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    #temp = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [ ]:
stopwords = nlp.Defaults.stop_words
def tokenize(x):
  doc=nlp(x)
  lst=[]
  for token in doc:
    if token.is_stop==False and token.is_punct==False:
      lst.append(token)
  lst1=[]
  for i in lst:
    i=str(i)
    lst1.append(i)

  return ' '.join(lst1)

df['tokenized']=df['Review'].apply(lambda x: tokenize(x))  

In [ ]:
clean_token = text_data_cleaning(df['Review'][1])
clean_token

['sure', 'lose', 'flat', 'character', 'audience', 'nearly', 'half', 'walk']

In [ ]:
clean2 = tokenize(df['Review'][1])
clean2

'sure lost flat characters audience nearly half walked  '

In [ ]:
X = df['Review']
y = df['Sentiment']

In [ ]:
df.head()

,Review,Sentiment,tokenized
0,"A very, very, very slow-moving, aimless movie ...",0,slow moving aimless movie distressed drifting ...
1,Not sure who was more lost - the flat characte...,0,sure lost flat characters audience nearly half...
2,Attempting artiness with black & white and cle...,0,Attempting artiness black white clever camera ...
3,Very little music or anything to speak of.,0,little music speak
4,The best scene in the movie was when Gerardo i...,1,best scene movie Gerardo trying find song keep...


In [ ]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
X = tfidf.fit_transform(df['Review'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 128)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(598, 2478)
(598,)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
classifier = LinearSVC()

classifier.fit(X_train,y_train)

LinearSVC()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))

[[59 18]
 [14 59]]
              precision    recall  f1-score   support

           0       0.81      0.77      0.79        77
           1       0.77      0.81      0.79        73

    accuracy                           0.79       150
   macro avg       0.79      0.79      0.79       150
weighted avg       0.79      0.79      0.79       150



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.64      0.71        77
           1       0.69      0.84      0.75        73

    accuracy                           0.73       150
   macro avg       0.74      0.74      0.73       150
weighted avg       0.75      0.73      0.73       150



In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.66      0.77        77
           1       0.73      0.95      0.82        73

    accuracy                           0.80       150
   macro avg       0.83      0.80      0.80       150
weighted avg       0.83      0.80      0.80       150



Sarcasm News

In [ ]:
!wget https://www.dropbox.com/s/ztu2gdjau2otq2a/Sarcasm_Headlines_Dataset.json

--2022-01-01 14:44:35--  https://www.dropbox.com/s/ztu2gdjau2otq2a/Sarcasm_Headlines_Dataset.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ztu2gdjau2otq2a/Sarcasm_Headlines_Dataset.json [following]
--2022-01-01 14:44:36--  https://www.dropbox.com/s/raw/ztu2gdjau2otq2a/Sarcasm_Headlines_Dataset.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc65c4783424e8734eacc804a51.dl.dropboxusercontent.com/cd/0/inline/Bc8XJRZ9BDQqj8YSEVls2yFw2NJYlTNZcdwpTiuYC9egXpQ99ihfmBBNJQ_jAbC_kJKdR7FJGCL4rcHI9z7IWgVmWF0Bqp6NmlyaSiXCpm_cpzyg-y6ykCK8IOoucdvuSPXaEb4nLB9kNsV7INbOY_Tg/file# [following]
--2022-01-01 14:44:36--  https://ucc65c4783424e8734eacc804a51.dl.dropboxusercontent.com/cd/0/inline/Bc8XJRZ9BDQqj8YSEVls2yFw2NJYlTNZc

In [ ]:
#article line , headline , is_sarcastic or not , .json
import pandas as pd
import spacy

In [ ]:
df = pd.read_json('/content/Sarcasm_Headlines_Dataset.json', lines=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
df.iloc[:, 2].value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [ ]:
df.isna().sum()

article_link    0
headline        0
is_sarcastic    0
dtype: int64

In [ ]:
df['num_words'] = df['headline'].str.split().str.len()

In [ ]:
df.head()

,article_link,headline,is_sarcastic,num_words
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0,12
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0,14
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1,14
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1,13
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0,11


In [ ]:
df['num_words'].describe()

count    26709.000000
mean         9.845820
std          3.168955
min          2.000000
25%          8.000000
50%         10.000000
75%         12.000000
max         39.000000
Name: num_words, dtype: float64

In [ ]:
text = df[df['num_words']==39]['headline'].values

In [ ]:
nlp = spacy.load('en')
doc = nlp(text[0])

In [ ]:
df_new = pd.DataFrame(
    {
    'token': [w.text for w in doc],
    'lemma':[w.lemma_ for w in doc],
    'POS': [w.pos_ for w in doc],
    'TAG': [w.tag_ for w in doc],
    'DEP': [w.dep_ for w in doc],
    'is_stopword': [w.is_stop for w in doc],
    'is_punctuation': [w.is_punct for w in doc],
    'is_digit': [w.is_digit for w in doc]       
    }
)

In [ ]:
df_new

,token,lemma,POS,TAG,DEP,is_stopword,is_punctuation,is_digit
0,elmore,elmore,PROPN,NNP,compound,False,False,False
1,leonard,leonard,NOUN,NN,nsubj,False,False,False
2,",",",",PUNCT,",",punct,False,True,False
3,modern,modern,ADJ,JJ,amod,False,False,False
4,prose,prose,NOUN,NN,compound,False,False,False
5,master,master,NOUN,NN,appos,False,False,False
6,",",",",PUNCT,",",punct,False,True,False
7,noted,note,VERB,VBD,ROOT,False,False,False
8,for,for,ADP,IN,prep,True,False,False
9,his,-PRON-,DET,PRP$,poss,True,False,False


In [ ]:
# as part of preprocessing Remove no.'s apart from funct and stop words which idea has come after looking at few of the dataset


In [ ]:
def clean_text(df):
  nlp = spacy.load('en')
  for i in range(df.shape[0]):
    doc = nlp(df['headline'][i])
    text = [w.lemma_.lower().strip() for w in doc 
            #if not (w.is_stop | w.is_punct | w.is_digit)]
            if not (w.is_stop | w.is_punct)]
    text = " ".join(text)

    if i<5:print('Sentence:', i,text)
    df['headline'][i] = text 

    return df

In [ ]:
news_df = clean_text(df)

Sentence: 0 versace store clerk sue secret black code minority shopper


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
news_df = clean_text(df)

Sentence: 0 versace store clerk sue secret black code minority shopper


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
news_df

,article_link,headline,is_sarcastic,num_words
0,https://www.huffingtonpost.com/entry/versace-b...,versace store clerk sue secret black code mino...,0,12
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0,14
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1,14
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1,13
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0,11
...,...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0,5
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0,4
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0,3
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0,8


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer() # default, arguments ===> assignment 

X = tf.fit_transform(news_df['headline'])

In [ ]:
y = news_df['is_sarcastic']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=64)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(X_train, y_train) 

BernoulliNB()

In [ ]:
y_pred = nb.predict(X_test)
from sklearn.metrics import classification_report 
print("NB classificatin results: ", classification_report(y_test, y_pred))

NB classificatin results:                precision    recall  f1-score   support

           0       0.84      0.91      0.87      3019
           1       0.86      0.77      0.82      2323

    accuracy                           0.85      5342
   macro avg       0.85      0.84      0.84      5342
weighted avg       0.85      0.85      0.85      5342



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_test)
print("RF classificatin results: ", classification_report(y_test, y_pred))

RF classificatin results:                precision    recall  f1-score   support

           0       0.85      0.81      0.83      3019
           1       0.77      0.81      0.79      2323

    accuracy                           0.81      5342
   macro avg       0.81      0.81      0.81      5342
weighted avg       0.81      0.81      0.81      5342



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [ ]:
y_pred = lr.predict(X_test)
print("LR classificatin results: ", classification_report(y_test, y_pred))

LR classificatin results:                precision    recall  f1-score   support

           0       0.85      0.86      0.86      3019
           1       0.82      0.81      0.81      2323

    accuracy                           0.84      5342
   macro avg       0.83      0.83      0.83      5342
weighted avg       0.84      0.84      0.84      5342



In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
lsvc = LinearSVC()
lsvc.fit(X_train,y_train)

LinearSVC()

In [ ]:
y_pred = lsvc.predict(X_test)
print("SVC classificatin results: ", classification_report(y_test, y_pred))

SVC classificatin results:                precision    recall  f1-score   support

           0       0.85      0.86      0.86      3019
           1       0.82      0.81      0.81      2323

    accuracy                           0.84      5342
   macro avg       0.84      0.83      0.83      5342
weighted avg       0.84      0.84      0.84      5342

